<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/eda/Global_climate_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

#### Global
* NASA's Global Climate Change Datasets
* NOAA's Climate Datasets
* WHO Air Pollution Data Portal
* Climate Change Knowledge Portal
* Climate Change Initiative
* Citizen Weather Observer Program (CWOP/APRS)

* https://ourworldindata.org/co2-and-greenhouse-gas-emissions
* https://github.com/owid/co2-data

#### Krakow
* Krakow temperatures
https://www.visualcrossing.com/weather/weather-data-services
* Krakow pollution levels
https://powietrze.gios.gov.pl/pjp/archives
* Air corridors
* https://reasonstobecheerful.world/stuttgart-ventilation-corridors-green-cool-air/
* What is direction and average speed of wind in Krakow?

# Questions
1. Temperature Changes:

  * How has the global average temperature changed over the years?
  * Are there specific regions where the temperature is rising faster?
  * What is the correlation between greenhouse gas concentration and global temperature rise?

2. Greenhouse Gas Emissions:
  * Which countries are the largest emitters of greenhouse gases, and how has this changed over time?
  * What sectors (energy, agriculture, industrial processes, etc.) contribute most to greenhouse gas emissions globally and locally?
  * How do per capita emissions vary across different countries?

3. Effects on Biodiversity:
  * How is climate change impacting biodiversity in different ecosystems?
  * Are certain species more affected than others?
  * What is the correlation between climate change and the extinction rate?

4. Climate Policies and Agreements:
  * How have different countries' emissions changed after major climate agreements (like the Paris Agreement)?
  * Are countries meeting their Nationally Determined Contributions (NDCs)?
  * How do the policies and efforts of different countries compare?

5. Extreme Weather Events:
  * Has the frequency of extreme weather events (like hurricanes, heatwaves, heavy rain) changed over time?
  * What regions are most affected by these extreme events?
  * What's the economic impact of these events?
  




* Temperature by city/country
  * https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data
  * Nasa dateset by country
* Emmisions by city/country
  * https://www.kaggle.com/datasets/thedevastator/global-fossil-co2-emissions-by-country-2002-2022
  * https://ourworldindata.org/co2-and-greenhouse-gas-emissions
  * https://github.com/owid/co2-data
* Krakow temperatures
  * https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/miesieczne/klimat/
* Krakow emmisions
  * https://powietrze.gios.gov.pl/pjp/archives

In [1]:
!pip install -U -q datasets
!pip install -U -q ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from ydata_profiling import ProfileReport

In [3]:
climate_dataset_krakow = load_dataset("vitaliy-sharandin/climate-krakow-monthly-1951-2023")['train'].to_pandas()
climate_dataset_global = load_dataset("vitaliy-sharandin/climate-global")['train'].to_pandas()
climate_dataset = load_dataset("vitaliy-sharandin/climate-global-by-country")['train'].to_pandas()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]